In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse('raw_data/0000 - cctns.xml')
root = tree.getroot()

In [3]:
root

<Element '{req_document.xsd}req_document' at 0x7f7ef0022c20>

In [4]:
for child in root:
    print(child.tag, child.attrib)

{req_document.xsd}title {}
{req_document.xsd}version {}
{req_document.xsd}p {'id': '1'}
{req_document.xsd}p {'id': '2'}
{req_document.xsd}p {'id': '3'}
{req_document.xsd}p {'id': '4'}
{req_document.xsd}p {'id': '5'}
